# Exercise: Parameter-efficient fine-tuning

Fine-tuning all parameters of pre-trained language models can be resource-intensive. Because of this, current research in natural language processing is looking into developing methods for adapting models to downstream tasks without full fine-tuning. These methods only tune a small number of model parameters while yielding performance comparable to that of a fully fine-tuned model.

In this exercise, you will implement LoRA, one of the most well-known methods for parameter-efficient fine-tuning. LoRA stands for “Low-Rank Adaptation of Large Language Models” and was originally described in a research article by [Hu et al. (2021)](https://arxiv.org/abs/2106.09685).

## Dataset

The data for this lab comes from the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/). The full dataset consists of 50,000 highly polar movie reviews collected from the Internet Movie Database (IMDB). Here, we use a random sample consisting of 2,000 reviews for training and 500 reviews for evaluation.

To load the dataset, we use the [Hugging Face Datasets](https://huggingface.co/docs/datasets/en/index) library.

In [1]:
from datasets import load_dataset

imdb_dataset = load_dataset('csv', data_files = {'train': 'train.csv', 'eval': 'eval.csv'})

imdb_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'review', 'label'],
        num_rows: 2000
    })
    eval: Dataset({
        features: ['index', 'review', 'label'],
        num_rows: 500
    })
})

As we can see, each sample in the dataset is a record with three fields: an internal index (`index`, an integer), the text of the review (`review`, a string), and the sentiment label (`label`, an integer – 1&nbsp;for “positive” and 0&nbsp;for “negative” sentiment).

Here is an example record:

In [2]:
imdb_dataset['train'][645]

{'index': 2981,
 'review': 'Brilliant execution in displaying once and for all, this time in the venue of politics, of how "good intentions do actually pave the road to hell". Excellent!',
 'label': 1}

## Tokeniser

As our pre-trained language model, we will use [DistilBERT](https://huggingface.co/docs/transformers/en/model_doc/distilbert), a compact encoder model with 40% less parameters than BERT base. DistilBERT is not actually a *large* language model by modern standards and thus does not benefit as much from parameter-efficient fine-tuning as other models. However, it has the benefit of being light and fast, and can be run even on consumer hardware.

To feed the movie reviews to DistilBERT, we need to tokenise them and encode the resulting tokens as integers in the model vocabulary. We start by loading the DistilBERT tokeniser using the [Auto classes](https://huggingface.co/docs/transformers/en/model_doc/auto):

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/home/x_andaf/.conda/envs/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

We then create a tokenised version of the dataset:

In [4]:
def tokenize_function(batch):
    return tokenizer(batch['review'], padding=True, truncation=True)

tokenized_imdb_dataset = imdb_dataset.map(tokenize_function, batched=True)

tokenized_imdb_dataset

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    eval: Dataset({
        features: ['index', 'review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

As we can see, tokenising adds two additional fields to each review: `input_ids` is the list of token ids corresponding to the review, and `attention_mask` is the list of indices specifying which tokens the encoder should attend to.

To avoid trouble when fine-tuning the model later, the next cell disables tokeniser parallelism.

In [5]:
import os

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

## Trainer

In this section, we will set up our workflow for training and evaluating DistilBERT models. The central component in this workflow is the [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer), which provides extensive configuration options. Here, we leave most of these options at their default value. Two changes we *do* make are to enable evaluation of the trained model after each epoch, and to log the training and evaluation loss after every 5&nbsp;training steps (the default is 500).

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='tmp_trainer',
    evaluation_strategy='epoch',
    logging_steps=5,
)

In addition to the loss, we also track classification accuracy. For this we import the [Hugging Face Evaluate](https://huggingface.co/docs/evaluate/en/index) library and define a small helper function `compute_metrics()` that the trainer will call after each epoch.

In [8]:
import evaluate

accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In the next cell we define a convenience function `make_trainer()` that creates a readily-configured trainer for a specified model (*model*). We will use this trainer both to train the model on the training section of the tokenised review dataset, and to evaluate it on the evaluation section.

In [9]:
from transformers import Trainer

def make_trainer(model):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_imdb_dataset['train'],
        eval_dataset=tokenized_imdb_dataset['eval'],
        compute_metrics=compute_metrics,
    )
    return trainer

## Problem 1: Full fine-tuning

In the rest of this notebook, we will work our way to the implementation of LoRA, and compare LoRA to traditional fine-tuning methods. Our first point of reference is a fully fine-tuned DistilBERT model.

We start by loading the pre-trained model:

In [10]:
from transformers import AutoModelForSequenceClassification

pretrained_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

pretrained_model

/home/x_andaf/.conda/envs/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

The architecture of DistilBERT is that of a standard Transformer encoder with an embedding layer (`embeddings`) followed by a stack of six Transformer blocks (`transformer`) and a feedforward network with two linear layers (`pre_classifier` and `classifier`) and a final dropout layer (`dropout`).

### Count the number of trainable parameters

One relevant measure in the context of parameter-efficient fine-tuning is the number of parameters that need to be changed when training a model. Your first task in this lab is to write a function `num_trainable_parameters()` that calculates this number for a given model.

In [11]:
def num_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

The function should implement the following specification:

> **num_trainable_parameters** (*model*)
>
> Returns the number of float-valued trainable parameters in the specified *model* as an integer.

#### 👍 Hint

The term *parameter* can refer to either complete tensors or the individual elements of these tensors. For example, a linear layer created by `nn.Linear(3, 5)` has 2&nbsp;tensor-valued parameters (a weight matrix and a bias vector) and 20&nbsp;float-valued parameters (the elements of these tensors). To get the tensor-valued parameters of a model, you can use the [`parameters()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters) method. A parameter is *trainable* if it requires gradient.

#### 🤞 Test your code

To test your code, apply your function to the pre-trained model. The correct number of float-valued trainable parameters for this model is 66,955,010.

In [12]:
print(num_trainable_parameters(pretrained_model))

66955010


### Fine-tuning

When we load the pre-trained model, the Hugging Face Transformers library warns us that the weights of the feedforward network have not yet been trained. To do so, we pass the pre-trained model to a trainer and initiate the fine-tuning process. Because full fine-tuning is so resource-intensive, we save the fine-tuned model to disk. Alternatively, we load an already fine-tuned version of the model (provided on the course website).

**⚠️ Please note that fine-tuning the model will take some time! ⚠️**

In [13]:
# Alternative A: Fine-tuning

finetuned_trainer = make_trainer(pretrained_model)
finetuned_trainer.train()
finetuned_trainer.save_model('finetuned')

# Alternative B: Load a fine-tuned model

# finetuned_model = AutoModelForSequenceClassification.from_pretrained('finetuned')

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.508200,0.343843,0.894000
2,0.015900,0.403192,0.912000
3,0.002100,0.476842,0.908000


### Convenience functions

Because we will repeat the steps we just took to fine-tune the pre-trained model several times in this notebook, we define two convenience functions:

In [14]:
def train(model):
    print('Number of trainable parameters:', num_trainable_parameters(model))
    trainer = make_trainer(model)
    trainer.train()
    return model

In [15]:
def evaluate(model):
    trainer = make_trainer(model)
    return trainer.evaluate()

## Problem 2: Tuning the final layers only

If full fine-tuning marks one end of the complexity spectrum, the other end is marked by only tuning the final layers of the transformer – the *head* of the model. In the case of DistilBERT, the head consists of the `pre_classifier` and `classifier` layers.

Implement the head-tuning strategy by coding the following function:

In [23]:
def make_headtuned_model():
    pretrained_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

    # Now we set `requires_grad` to `False` for every layer apart from the `classifier` and `pre_classifier`
    for name, p in pretrained_model.named_parameters():
        if "pre_classifier" not in name and "classifier" not in name:
            p.requires_grad = False

    return pretrained_model

Here is the specification of this function:

> **make_headtuned_model** ()
>
> Returns a model that is identical to the pre-trained model, except that the head layers have been trained on the sentiment data. (The other parameters of the pre-trained model are left untouched.)

#### 👍 Hint

You freeze a parameter by setting its `requires_grad`-attribute to `False`.

Once you have an implementation of the head-tuning strategy, evaluate it on the evaluation data. How much accuracy do we lose when only training the final layers of the pre-trained model, compared to full fine-tuning?

In [29]:
headtuned_model = make_headtuned_model()

/home/x_andaf/.conda/envs/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 🤞 Test your code

If you configured your model correctly, `num_trainable_parameters()` should show 592,130 trainable parameters.

In [30]:
print(num_trainable_parameters(headtuned_model))

592130


For future reference, we also save the head-tuned model:

In [33]:
train(headtuned_model)
evaluate(headtuned_model)
make_trainer(headtuned_model).save_model("headtuned")

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Number of trainable parameters: 592130


Epoch,Training Loss,Validation Loss,Accuracy
1,0.444100,0.460707,0.806000
2,0.316000,0.430408,0.818000
3,0.408700,0.414920,0.822000


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Problem 3: Layer surgery

LoRA works by “wrapping” frozen layers from the pre-trained Transformer model inside adapter modules. Conventionally, this wrapping is only applied to the linear layers that transform the queries and values in the self-attention mechanism. To implement the wrapping, we need functions to extract and replace layers in a model. Your task in this section is to code these functions.

### Extracting layers

Code a function that extracts the query and value linear layers from a DistilBERT model:

In [56]:
def extract(model):
    return {name: layer for name, layer in model.named_modules() if "q_lin" in name or "v_lin" in name}

Implement this function to match the following specification:

> **extract** (*model*)
>
> Takes a DistilBERT model (*model*) and extracts the query and value linear layers from each block of the Transformer. Returns a dictionary mapping the DistilBERT module names of these layers to the layers themselves (instances of `nn.Linear`).

#### 👍 Hint

As we saw earlier, the DistilBERT model consists of a hierarchy of nested submodules. Each of these can be addressed by a fully-qualified string name. Use [`get_submodule()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.get_submodule) to retrieve a layer by name. You can hard-wire the names of the layers you want to extract.

#### 🤞 Test your code

To test your code, check the number of trainable float-valued parameters in the extracted layers. This number should be 7,087,104.

In [57]:
pretrained_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
extract(pretrained_model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'distilbert.transformer.layer.0.attention.q_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.0.attention.v_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.1.attention.q_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.1.attention.v_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.2.attention.q_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.2.attention.v_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.3.attention.q_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.3.attention.v_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.4.attention.q_lin': Linear(in_features=768, out_features=768, bias=True),
 'distilbert.transformer.layer.4.attention.v_lin': Line

In [58]:
trainable_parameters = 0
for layer in extract(pretrained_model).values():
    trainable_parameters += num_trainable_parameters(layer)
print(trainable_parameters)

7087104


### Replacing layers

The inverse of the `extract()` function replaces selected layers of a module using a dictionary of named layers.

In [59]:
def replace(model, named_layers):
    for name, layer in named_layers.items():
        components = name.split('.')
        submodule = model
        for component in components[:-1]:
            submodule = getattr(submodule, component)
        setattr(submodule, components[-1], layer)
    return model

This function has the following specification:

> **replace** (*model*, *named_layers*)
>
> Takes a DistilBERT model (*model*) and a dictionary in the format returned by `extract()` (*named_layers*) and injects the extracted layers into the model. More specifically, suppose that *named_layers* contains a key–value pair `(name, layer)`. Then the function replaces the submodule of *model* addressed by the fully-qualified string name `name` by the layer `layer`. Returns the modified model.

#### 🤞 Test your code

To test your implementation, write code that (1)&nbsp;extracts the query and value linear layers from the fine-tuned model; (2)&nbsp;replaces these layers with clones with random weights; and (3)&nbsp;replaces these layers again with the original versions. Evaluating the modified model after step&nbsp;(2) should yield a near-random accuracy. Evaluating it again after step&nbsp;(3) should yield the original accuracy.

The following function should be helpful. It clones a linear layer, copying the weights and the bias from the original.

In [63]:
import torch.nn as nn

def clone_linear(original):
    out_features, in_features = original.weight.shape
    copy = nn.Linear(in_features, out_features)
    copy.load_state_dict(original.state_dict())
    return copy

In [74]:
import torch.nn as nn

# 1. Extract the query and values linear layers
extracted_layers = extract(finetuned_trainer.model)

# 2. Replace layers with clones with random weights
random_clone_layers = {name: clone_linear(layer) for name, layer in extracted_layers.items()}
for layer in random_clone_layers.values():
    nn.init.normal_(layer.weight)
replace(finetuned_trainer.model, random_clone_layers)
print(evaluate(finetuned_trainer.model))

# 3. Replace back the original layers
replace(finetuned_trainer.model, extracted_layers)
print(evaluate(finetuned_trainer.model))

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'eval_loss': 0.6988790035247803, 'eval_accuracy': 0.502, 'eval_runtime': 2.1754, 'eval_samples_per_second': 229.848, 'eval_steps_per_second': 28.961}


{'eval_loss': 0.4768417179584503, 'eval_accuracy': 0.908, 'eval_runtime': 2.1261, 'eval_samples_per_second': 235.173, 'eval_steps_per_second': 29.632}


## Problem 4: LoRA

In this section, you will implement the LoRA adapters and fine-tune the adapted model.

Recall that the basic idea behind LoRA is to conceptualise fine-tuned weights as a sum $W_0 + \Delta W$ of the weights from the pre-trained model, $W_0$, and a low-rank update matrix $\Delta W$. The goal of fine-tuning, then, is to learn the update matrix; this happens in the adapter layers.

### Implement the adapter

A LoRA adapter implements the forward function

$$
y = x W_0 + x \Delta W = x W_0 + x A B
$$

where $W_0$ is a linear transformation from the pre-trained model and $\Delta W$ is a learned update matrix, deconstructed into the product $AB$ of two rank-$r$ matrices $A$ and $B$. LoRA scales the update matrix $\Delta W$ by a factor of $\alpha / r$, where $\alpha$ is a hyperparameter. (To keep the formula tidy, we ignore the fact that the linear transformation in the pre-trained model may additionally include a bias.)

In [103]:
import torch.nn as nn

class LoRA(nn.Module):

    def __init__(self, pretrained, rank=12, alpha=24):
        super().__init__()
        self.base_layer = pretrained
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank
        self.lora_a = nn.Linear(pretrained.in_features, rank, bias=False)
        self.lora_b = nn.Linear(rank, pretrained.out_features, bias=False)

    def forward(self, x):
        return self.base_layer(x) + self.scaling * (self.lora_b(self.lora_a(x)))

Your code must comply with the following specification:

**__init__** (*self*, *pretrained*, *rank* = 12, *alpha* = 24)

> Initialises the LoRA adapter. This sets up the matrices $A$ and $B$ from the equation above. The matrix $A$ is initialised with random weights from a standard normal distribution; the matrix $B$ is initialised with zeros. The argument *pretrained* is the linear layer from the pre-trained model that should be adapted. The arguments *rank* and *alpha* are the rank $r$ and the hyperparameter $\alpha$ in the equation above.

**forward** (*self*, *x*)

> Sends an input *x* through the adapter, implementing the equation above.

### Inject the adapter into the pre-trained model

The final step is to construct an adapted model by injecting the LoRA adapters into the pre-trained model.

In [104]:
def make_lora_model(rank, alpha):
    # First, we create the model and extract the layers we want
    model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
    extracted_layers = extract(model)
    # Then we create the LoRA layers and replace the original ones by them
    lora_layers = {name: LoRA(layer, rank, alpha) for name, layer in extracted_layers.items()}
    replace(model, lora_layers)
    # Finally we set the 
    for name, p in model.named_parameters():
        if "lora_a" in name or "lora_b" in name or "classifier" in name or "preclassifier" in name:
            p.requires_grad = True
        else:
            p.requires_grad = False
    return model

Implement the function to match the following specification:

> **make_lora_model** (*rank*)
>
> Returns a model that is identical to the pre-trained model, except that the query and value linear layers have been wrapped in LoRA adapters, and the LoRA adapters and the head layers of the pre-trained model have been trained on the sentiment data. (The other parameters of the pre-trained model are left untouched.) The rank of the adapters is specified by the argument *rank*. The *alpha* value of the adapters is set to twice the rank (a common rule of thumb).

Run the next cell to evaluate your model for $r = 6$ and $\alpha = 12$. How many trainable parameters does the adapted model have? What accuracy do you get? How do these value relate to the number of trainable parameters and accuracy of the fully fine-tuned model, in terms of percentages?

In [105]:
lora_model = make_lora_model(6, 12)
print(lora_model)

print(num_trainable_parameters(lora_model))

print(evaluate(lora_model))

train(lora_model)

print(evaluate(lora_model))

/home/x_andaf/.conda/envs/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): LoRA(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_a): Linear(in_features=768, out_features=6, bias=False)
              (lora_b): Linear(in_features=6, out_features=768, bias=False)
            )
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): LoRA(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
           

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'eval_loss': 0.6923620700836182, 'eval_accuracy': 0.512, 'eval_runtime': 2.3032, 'eval_samples_per_second': 217.093, 'eval_steps_per_second': 27.354}
Number of trainable parameters: 702722


Epoch,Training Loss,Validation Loss,Accuracy
1,0.508700,0.473535,0.806000
2,0.248900,0.353711,0.850000
3,0.341400,0.340413,0.854000


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'eval_loss': 0.34041333198547363, 'eval_accuracy': 0.854, 'eval_runtime': 2.1991, 'eval_samples_per_second': 227.363, 'eval_steps_per_second': 28.648}


## Problem 5: Alternatives to Transformer-based models

Even with methods for parameter-efficient fine-tuning, applying DistilBERT and other Transformer-based models comes at a significant cost – an investment that does not always pay off. In the final problem of this lab, we ask you to explore a more traditional approach to classification and contrast it with the pre-training/fine-tuning approach of neural language models.

Browse the web to find a tutorial on how to apply a classifier from the [scikit-learn](https://scikit-learn.org/stable/) library to the problem of sentiment classification and implement the method here in this notebook. We suggest you use [multinomial Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) or [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). (Once you have code for one method, it is easy to switch to the other.) Evaluate your chosen classifier on the IMDB dataset.

* Which classifier did you try? What results did you get? How long did it take you to train and run the classifier?
* What is your perspective on the trade-off between accuracy and resource requirements between the two approaches?